# リッジ回帰を行う

## 準備

In [1]:
import pandas as pd
import numpy as np
import glob
from sklearn.linear_model import Ridge

In [2]:
stock_name_list = glob.glob("../output/*.csv")
stock_name_list

['../output\\LSIML_RV_mitsuifin_1sec.csv',
 '../output\\LSIML_RV_mitsuifin_5sec.csv']

In [3]:
df = pd.read_csv(stock_name_list[0])
df

,date,RV_sec1,LSIML_b=10,LSIML_b=50,LSIML_b=100,Num_jump_b=10,Num_jump_b=50,Num_jump_b=100,Size_jump_b=10,Size_jump_b=50,Size_jump_b=100
0,2018-09-03,0.000100,0.000034,0.000046,0.000046,1,4,6,0.000012,0.000024,0.000025
1,2018-09-04,0.000152,0.000040,0.000041,0.000047,1,5,9,0.000027,0.000028,0.000032
2,2018-09-05,0.000086,0.000025,0.000031,0.000036,1,4,8,0.000014,0.000017,0.000018
3,2018-09-06,0.000094,0.000038,0.000034,0.000039,0,4,6,0.000000,0.000011,0.000011
4,2018-09-07,0.000123,0.000039,0.000055,0.000044,2,3,14,0.000044,0.000023,0.000038
...,...,...,...,...,...,...,...,...,...,...,...
699,2021-07-26,0.000125,0.000025,0.000035,0.000046,1,4,5,0.000018,0.000016,0.000018
700,2021-07-27,0.000091,0.000023,0.000024,0.000031,1,4,5,0.000012,0.000013,0.000011
701,2021-07-28,0.000105,0.000031,0.000035,0.000044,1,6,7,0.000011,0.000019,0.000015
702,2021-07-29,0.000089,0.000027,0.000036,0.000035,1,4,9,0.000012,0.000019,0.000019


## データを読み込み、加工する

In [4]:
LSIML = df["LSIML_b=50"] + df["Size_jump_b=50"]
LSIML

0      0.000070
1      0.000069
2      0.000048
3      0.000045
4      0.000078
         ...   
699    0.000051
700    0.000037
701    0.000054
702    0.000055
703    0.000047
Length: 704, dtype: float64

### 移動平均, 一日ずれデータを計算する

In [5]:
week = LSIML.rolling(5).mean().shift()
month = LSIML.rolling(22).mean().shift()

In [6]:
month.dropna()

22     0.000075
23     0.000077
24     0.000083
25     0.000085
26     0.000089
         ...   
699    0.000053
700    0.000050
701    0.000050
702    0.000049
703    0.000050
Length: 682, dtype: float64

In [7]:
shifted_LSIML = LSIML.shift()
shifted_LSIML

0           NaN
1      0.000070
2      0.000069
3      0.000048
4      0.000045
         ...   
699    0.000057
700    0.000051
701    0.000037
702    0.000054
703    0.000055
Length: 704, dtype: float64

In [8]:
data = pd.concat([LSIML, shifted_LSIML, week, month], axis=1)
data.columns = ["tom", "tod", "week", "month"]
data

,tom,tod,week,month
0,0.000070,NaN,NaN,NaN
1,0.000069,0.000070,NaN,NaN
2,0.000048,0.000069,NaN,NaN
3,0.000045,0.000048,NaN,NaN
4,0.000078,0.000045,NaN,NaN
...,...,...,...,...
699,0.000051,0.000057,0.000054,0.000053
700,0.000037,0.000051,0.000055,0.000050
701,0.000054,0.000037,0.000052,0.000050
702,0.000055,0.000054,0.000055,0.000049


### 対数を取る

In [9]:
data["log_tom"] = np.log(data["tom"])
data["log_tod"] = np.log(data["tod"])
data["log_week"] = np.log(data["week"])
data["log_month"] = np.log(data["month"])
data

,tom,tod,week,month,log_tom,log_tod,log_week,log_month
0,0.000070,NaN,NaN,NaN,-9.572020,NaN,NaN,NaN
1,0.000069,0.000070,NaN,NaN,-9.582794,-9.572020,NaN,NaN
2,0.000048,0.000069,NaN,NaN,-9.953234,-9.582794,NaN,NaN
3,0.000045,0.000048,NaN,NaN,-10.009332,-9.953234,NaN,NaN
4,0.000078,0.000045,NaN,NaN,-9.454951,-10.009332,NaN,NaN
...,...,...,...,...,...,...,...,...
699,0.000051,0.000057,0.000054,0.000053,-9.874341,-9.766005,-9.822059,-9.836261
700,0.000037,0.000051,0.000055,0.000050,-10.203925,-9.874341,-9.812816,-9.895534
701,0.000054,0.000037,0.000052,0.000050,-9.822236,-10.203925,-9.865343,-9.908713
702,0.000055,0.000054,0.000055,0.000049,-9.805023,-9.822236,-9.815572,-9.923729


### ジャンプサイズ

In [10]:
data["jump"] = df["Size_jump_b=50"].shift()

### 日次データを読み込む

In [11]:
daily_data = pd.read_csv("../daily_data\Daily_Price_mitsuifin.csv").shift()
daily_data

,date,log_price,daily_return
0,NaN,NaN,NaN
1,2018-09-03,8.381144,NaN
2,2018-09-04,8.377701,-0.003443
3,2018-09-05,8.373092,-0.004609
4,2018-09-06,8.373092,0.000000
...,...,...,...
699,2021-07-21,8.209852,0.001361
700,2021-07-26,8.214194,0.004342
701,2021-07-27,8.226306,0.012112
702,2021-07-28,8.222016,-0.004290


In [12]:
asym_list = []
for i in range(len(daily_data)):
    asym_list.append(min([0, daily_data["daily_return"].iloc[i]]))

asym_list

[0,
 0,
 -0.0034431343553222,
 -0.0046093651556589,
 0,
 -0.0041666726948452,
 0,
 -0.0009233610997689,
 -0.0041657084077009,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.0240871860382654,
 -0.0107233766374648,
 0,
 0,
 -0.0002165674074507,
 -0.0170384200505235,
 0,
 0,
 -0.0121446043640141,
 0,
 -0.0422536721838646,
 -0.013212598435464,
 -0.020267117481465,
 0,
 0,
 -0.0022341385521613,
 -0.0047080010736948,
 0,
 -0.0194840780832414,
 -0.015446993021694,
 -0.0173428298012954,
 0,
 -0.0061393345135094,
 0,
 0,
 0,
 0,
 -0.0033978965754055,
 0,
 -0.0056173611367302,
 0,
 0,
 -0.0042114658279288,
 -0.0274730180961846,
 0,
 -0.0309480286208252,
 -0.0051353987743301,
 -0.0212874642018263,
 0,
 -0.0088633949516445,
 -0.0021678919727357,
 -0.0041077742913024,
 0,
 0,
 -0.0019093084559198,
 -0.0007169315637458,
 0,
 -0.0276692738597965,
 -0.0046460531061303,
 -0.01158647252484,
 -0.0074664358807936,
 -0.0102951317340771,
 -0.0162853471909905,
 0,
 0,
 -0.0068693820566956,
 0,
 -0.0163812211834581,
 -0.006

In [13]:
data["asym"] = asym_list

### 欠損値を削除する

In [14]:
data = data.dropna()
data

,tom,tod,week,month,log_tom,log_tod,log_week,log_month,jump,asym
22,0.000105,0.000183,0.000099,0.000075,-9.158795,-8.605646,-9.215656,-9.492234,0.000133,0.000000
23,0.000200,0.000105,0.000104,0.000077,-8.515403,-9.158795,-9.169639,-9.470987,0.000052,0.000000
24,0.000100,0.000200,0.000130,0.000083,-9.213129,-8.515403,-8.945842,-9.396302,0.000070,-0.012145
25,0.000129,0.000100,0.000139,0.000085,-8.957312,-9.213129,-8.878672,-9.368154,0.000027,0.000000
26,0.000103,0.000129,0.000143,0.000089,-9.179210,-8.957312,-8.849551,-9.324511,0.000048,-0.042254
...,...,...,...,...,...,...,...,...,...,...
699,0.000051,0.000057,0.000054,0.000053,-9.874341,-9.766005,-9.822059,-9.836261,0.000024,0.000000
700,0.000037,0.000051,0.000055,0.000050,-10.203925,-9.874341,-9.812816,-9.895534,0.000016,0.000000
701,0.000054,0.000037,0.000052,0.000050,-9.822236,-10.203925,-9.865343,-9.908713,0.000013,0.000000
702,0.000055,0.000054,0.000055,0.000049,-9.805023,-9.822236,-9.815572,-9.923729,0.000019,-0.004290


# リッジ回帰を行う

In [15]:
data

,tom,tod,week,month,log_tom,log_tod,log_week,log_month,jump,asym
22,0.000105,0.000183,0.000099,0.000075,-9.158795,-8.605646,-9.215656,-9.492234,0.000133,0.000000
23,0.000200,0.000105,0.000104,0.000077,-8.515403,-9.158795,-9.169639,-9.470987,0.000052,0.000000
24,0.000100,0.000200,0.000130,0.000083,-9.213129,-8.515403,-8.945842,-9.396302,0.000070,-0.012145
25,0.000129,0.000100,0.000139,0.000085,-8.957312,-9.213129,-8.878672,-9.368154,0.000027,0.000000
26,0.000103,0.000129,0.000143,0.000089,-9.179210,-8.957312,-8.849551,-9.324511,0.000048,-0.042254
...,...,...,...,...,...,...,...,...,...,...
699,0.000051,0.000057,0.000054,0.000053,-9.874341,-9.766005,-9.822059,-9.836261,0.000024,0.000000
700,0.000037,0.000051,0.000055,0.000050,-10.203925,-9.874341,-9.812816,-9.895534,0.000016,0.000000
701,0.000054,0.000037,0.000052,0.000050,-9.822236,-10.203925,-9.865343,-9.908713,0.000013,0.000000
702,0.000055,0.000054,0.000055,0.000049,-9.805023,-9.822236,-9.815572,-9.923729,0.000019,-0.004290


元系列に対して、非対称性が存在するか確かめる

In [45]:
x_names = ["tod", "week", "month", "asym", "jump"]
ridge = Ridge().fit(data[x_names], data["tom"])
# 係数を出力する
res_coef = pd.DataFrame(ridge.coef_).T 
res_coef.columns = x_names
res_coef

,tod,week,month,asym,jump
0,0.000015,0.000012,0.000006,-0.000388,0.000005


対数系列に対して、非対称性が存在するか確かめる

In [49]:
x_names = ["log_tod", "log_week", "log_month", "asym", "jump"]
ridge = Ridge().fit(data[x_names], data["log_tom"])
# 係数を出力する
res_coef = pd.DataFrame(ridge.coef_).T 
res_coef.columns = x_names
res_coef

,log_tod,log_week,log_month,asym,jump
0,0.460301,0.357776,0.098592,-0.5741,-0.000803
